In [1]:
## Data Transformation - County City Lookup
## Version 6
### 1/12/25
### 21:25

In [2]:
import sys
import os

# Add the root directory to sys.path
sys.path.append(os.path.abspath(".."))

# Now you can use absolute imports
from d497_helpers import folder_manager as fm, archive_module, checkpoint_helper as CheckPoint, database_helper as db_tool, config

# Pandas
import pandas as pd

# Numpy
import numpy as np

# RegularExpression for string matching
import re

# GLOB 
import glob 

# Pathlib 
from pathlib import Path


In [3]:
fips_xr_city_df = pd.DataFrame()
fips_xr_county_city_df = pd.DataFrame()

In [4]:

# Initializes Database Engine and Session For SQL
db_tool.initialize_engine()
db_tool.initialize_session()


In [5]:
city_lookup_sql_query = '''
    SELECT
        state_fipcode,
        city_name AS city_lookup,
        city_fipcode AS city_code
    FROM 
        fips_data 
        '''

In [6]:
fips_xr_city_df = db_tool.export_df_from_sql(city_lookup_sql_query, fips_xr_city_df)

2025-01-20 17:52:29,458 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-20 17:52:29,459 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("
    SELECT
        state_fipcode,
        city_name AS city_lookup,
        city_fipcode AS city_code
    FROM 
        fips_data 
        ")
2025-01-20 17:52:29,461 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 17:52:29,463 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("
    SELECT
        state_fipcode,
        city_name AS city_lookup,
        city_fipcode AS city_code
    FROM 
        fips_data 
        ")
2025-01-20 17:52:29,464 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 17:52:29,465 INFO sqlalchemy.engine.Engine 
    SELECT
        state_fipcode,
        city_name AS city_lookup,
        city_fipcode AS city_code
    FROM 
        fips_data 
        
2025-01-20 17:52:29,466 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 17:52:29,539 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
fips_xr_city_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39791 entries, 0 to 39790
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   state_fipcode  39791 non-null  object
 1   city_lookup    39791 non-null  object
 2   city_code      39791 non-null  object
dtypes: object(3)
memory usage: 932.7+ KB


In [8]:
# Show the number of duplicate rows, but does not include the first row found. 
# i.e. These are the rows that have been labled as duplicates to other rows in the dataset

fips_xr_city_df.duplicated().value_counts()

False    38714
True      1077
Name: count, dtype: int64

In [9]:
# By appling the "keep=False" arguement to the duplicated function, .duplicated()
# We are now able to capture all rows that have been marked as a duplicate and the 
# corresponding row that they were matched to. 

fips_xr_city_df.duplicated(keep=False).value_counts()

False    37703
True      2088
Name: count, dtype: int64

In [10]:
# Currently, it appears there may be some duplicates in the dataset that will need
# to be investigated before we can move forward. The data we utilized was the state_code,
# city_name, and city_code. The duplicated function indicates there are 2088 rows that
# are duplicates (2 or more for each match including the original row)

# Let's start by visually inspecting the results of the duplicated function, includin 
# the first matched row, and see if we can verify these results. We will do this with 
# a loc call on the dataset using the duplicated(keep=False) as our filter. To make the 
# review easier, we will also sort the values by state_code and then city_lookup (city_name).
# We do not need to review all 2088. So let's just get the first 20 rows from the dataset 
# using the head function. 


In [11]:
fips_xr_city_df.loc[fips_xr_city_df.duplicated(keep=False)].sort_values(by=["state_fipcode","city_lookup"]).head(20)

,state_fipcode,city_lookup,city_code
271,01,BOAZ,0400
522,01,BOAZ,0400
365,01,BROWNVILLE,0506
699,01,BROWNVILLE,0506
50,01,COUNTY LINE,0798
373,01,COUNTY LINE,0798
379,01,FLAT CREEK,1170
723,01,FLAT CREEK,1170
165,01,FORT RUCKER,1235
229,01,FORT RUCKER,1235


In [12]:
# With what we can see/what data we currently have available to us, it does
# appear that the rows are full duplicates. This is the obvious answer, as the 
# duplicated function will only match the entire row unless passed additional 
# arguments, which we did not do. 

# Let's expand on the data we have available and see if we can isolate where this
# duplication issue occurred. We know that from our sql query forward that the issue 
# occurs. We also know that the source data and our extraction from it's source location
# is valid and accurate. This is because the data comes from a federally maintained 
# API. 

# Instead of using another sql query and pandas to verify the data in our database, 
# I am going to check the data using a free, datahase tool called, DB Browser for SQLite.

# Tool Website
# https://sqlitebrowser.org/

# download can be found here:
# https://sqlitebrowser.org/dl/


# Using this tool, I was successfully able to open and view the data in our datbase. 
# I then performed a sort on the city_name data, alphabetically. Next, using one of the 
# duplicated rows we captured above, I navigated to the data until I found the city name 
# that matched and then checked for other similar items. I was able to confirm there are 
# multiple cities with the same name and the same city code in our dataset. I was also 
# able to confirm that there are multiple cities with the same name and the same state 
# in our database. Fortunately, I was also able to confirm that each of these entries 
# are from a different county in the same state. This means that the current sql query 
# we have will not work and we will need to expand it to include the county information. 


In [13]:
del fips_xr_city_df

In [14]:
county_city_lookup_sql_query = '''
    SELECT
        state_fipcode,
        county_name AS county_lookup,
        county_fipcode AS county_fipcode,
        city_name AS city_lookup,
        city_fipcode AS city_fipcode
    FROM 
        fips_data
        '''

In [15]:
fips_xr_county_city_df = db_tool.export_df_from_sql(county_city_lookup_sql_query, fips_xr_county_city_df)

2025-01-20 17:52:29,666 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-20 17:52:29,668 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("
    SELECT
        state_fipcode,
        county_name AS county_lookup,
        county_fipcode AS county_fipcode,
        city_name AS city_lookup,
        city_fipcode AS city_fipcode
    FROM 
        fips_data
        ")
2025-01-20 17:52:29,669 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 17:52:29,670 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("
    SELECT
        state_fipcode,
        county_name AS county_lookup,
        county_fipcode AS county_fipcode,
        city_name AS city_lookup,
        city_fipcode AS city_fipcode
    FROM 
        fips_data
        ")
2025-01-20 17:52:29,671 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 17:52:29,673 INFO sqlalchemy.engine.Engine 
    SELECT
        state_fipcode,
        county_name AS county_lookup,
        county_fipcode AS county_fipcode,
        city_name AS

In [16]:
fips_xr_county_city_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39791 entries, 0 to 39790
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   state_fipcode   39791 non-null  object
 1   county_lookup   39791 non-null  object
 2   county_fipcode  39791 non-null  object
 3   city_lookup     39791 non-null  object
 4   city_fipcode    39791 non-null  object
dtypes: object(5)
memory usage: 1.5+ MB


In [17]:
fips_xr_county_city_df.head(20)

,state_fipcode,county_lookup,county_fipcode,city_lookup,city_fipcode
0,01,AUTAUGA,C001,AUTAUGAVILLE,0220
1,01,AUTAUGA,C001,BILLINGSLEY,0340
2,01,AUTAUGA,C001,BOOTH,0425
3,01,AUTAUGA,C001,JONES,1796
4,01,AUTAUGA,C001,MARBURY,2002
5,01,AUTAUGA,C001,PRATTVILLE,2540
6,01,BALDWIN,C003,BAY MINETTE,0240
7,01,BALDWIN,C003,BON SECOUR,0423
8,01,BALDWIN,C003,DAPHNE,0860
9,01,BALDWIN,C003,ELBERTA,1011


In [18]:
fips_xr_county_city_df.duplicated().value_counts()

False    39790
True         1
Name: count, dtype: int64

In [19]:
fips_xr_county_city_df.loc[fips_xr_county_city_df.duplicated(keep=False)]

,state_fipcode,county_lookup,county_fipcode,city_lookup,city_fipcode
5872,12,BREVARD,C009,GRANT-VALKARIA,1188
5873,12,BREVARD,C009,GRANT-VALKARIA,1188


In [20]:
# Using the database tool again, I was able to confirm this row is a true duplicate. 
# We can drop it without worry

fips_xr_county_city_df.drop_duplicates(inplace=True)

In [21]:
fips_xr_county_city_df.duplicated().value_counts()

False    39790
Name: count, dtype: int64

In [22]:
fips_xr_county_city_df.sort_values(by="city_lookup", inplace=True)

In [23]:
fips_xr_county_city_df.reset_index()

,index,state_fipcode,county_lookup,county_fipcode,city_lookup,city_fipcode
0,29392,42,CENTRE,C027,AARONSBURG,0005
1,6719,13,WILCOX,C315,ABBEVILLE,0010
2,18117,28,LAFAYETTE,C071,ABBEVILLE,0005
3,31330,45,ABBEVILLE,C001,ABBEVILLE,0010
4,319,01,HENRY,C067,ABBEVILLE,0010
...,...,...,...,...,...,...
39785,11583,20,ROOKS,C163,ZURICH,6030
39786,19877,30,BLAINE,C005,ZURICH,1354
39787,10217,19,DUBUQUE,C061,ZWINGLE,9340
39788,10399,19,JACKSON,C097,ZWINGLE,9340


In [24]:
fips_xr_county_city_df.set_index('city_lookup')

,state_fipcode,county_lookup,county_fipcode,city_fipcode
city_lookup,,,,
AARONSBURG,42,CENTRE,C027,0005
ABBEVILLE,13,WILCOX,C315,0010
ABBEVILLE,28,LAFAYETTE,C071,0005
ABBEVILLE,45,ABBEVILLE,C001,0010
ABBEVILLE,01,HENRY,C067,0010
...,...,...,...,...
ZURICH,20,ROOKS,C163,6030
ZURICH,30,BLAINE,C005,1354
ZWINGLE,19,DUBUQUE,C061,9340


In [25]:
fips_xr_county_city_df.isnull().sum()

state_fipcode     0
county_lookup     0
county_fipcode    0
city_lookup       0
city_fipcode      0
dtype: int64

In [26]:
fips_xr_county_city_df.loc[fips_xr_county_city_df['city_fipcode'].isnull()]

,state_fipcode,county_lookup,county_fipcode,city_lookup,city_fipcode


In [27]:
fips_xr_county_city_df.isnull().sum()

state_fipcode     0
county_lookup     0
county_fipcode    0
city_lookup       0
city_fipcode      0
dtype: int64

In [28]:
db_tool.export_to_sql(fips_xr_county_city_df, "county_city_lookup")

2025-01-20 17:52:30,122 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-20 17:52:30,136 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("county_city_lookup")
2025-01-20 17:52:30,137 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 17:52:30,138 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("county_city_lookup")
2025-01-20 17:52:30,139 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 17:52:30,141 INFO sqlalchemy.engine.Engine 
CREATE TABLE county_city_lookup (
	state_fipcode TEXT, 
	county_lookup TEXT, 
	county_fipcode TEXT, 
	city_lookup TEXT, 
	city_fipcode TEXT
)


2025-01-20 17:52:30,142 INFO sqlalchemy.engine.Engine [no key 0.00090s] ()
2025-01-20 17:52:34,726 INFO sqlalchemy.engine.Engine INSERT INTO county_city_lookup (state_fipcode, county_lookup, county_fipcode, city_lookup, city_fipcode) VALUES (?, ?, ?, ?, ?)
2025-01-20 17:52:34,727 INFO sqlalchemy.engine.Engine [generated in 0.14471s] [('42', 'CENTRE', 'C027', 'AARONSBURG', '0005'), ('13', 'WILCOX

In [29]:
fips_xr_county_city_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39790 entries, 29392 to 13556
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   state_fipcode   39790 non-null  object
 1   county_lookup   39790 non-null  object
 2   county_fipcode  39790 non-null  object
 3   city_lookup     39790 non-null  object
 4   city_fipcode    39790 non-null  object
dtypes: object(5)
memory usage: 1.8+ MB


In [30]:
#fips_xr_county_city_df_file_name_pickle = "county_city_lookup.pkl"
#fips_xr_county_city_df_file_name_csv = "county_city_lookup.csv"

# Exporiting CSV as well to be on the safe side. 
#fips_xr_county_city_df.to_csv(cleaned_fips_data_folder_path + "/" + fips_xr_county_city_df_file_name_csv, index=False)

# Exporting as Pickle instead of CSV to perserve data types. 
#fips_xr_county_city_df.to_pickle(cleaned_fips_data_folder_path + "/" + fips_xr_county_city_df_file_name_pickle)


In [31]:
CheckPoint.create_checkpoint("cleaned_fips_data", "county_city_lookup", fips_xr_county_city_df)

In [32]:
del fips_xr_county_city_df